# Code Generation and Evaluation

__TABLE 1. AVAILABLE MISTRAL AI MODELS__
| Name                        | # Parameters | # Active Parameters | Min. GPU RAM for inference (GB) | API Endpoint          | Comments   |   Official HumanEval  |  HumanEval Full Func  | HumanEval Completion | MBPP |
| --------------------------- | ------------ | ------------------- | ------------------------------- | --------------------- | ------------------------------- |  ---------------  |  ---------------------  | ----------- | --------- |
| __MODELS I PLAN TO USE:__ |
| Mistral-7B-v0.3             | 7.3B         | 7.3B                | 16                              | open-mistral-7b       |                     |  30.5%  | 31.1%      |    |  13.6%  |
| Mixtral-8x7B-v0.1           | 46.7B        | 12.9B               | 100                             | open-mixtral-8x7b     |       |  40.2%  | 16.46%  |      |   0%    |
| Ministral-3B-2410           | 3B           | 3B                  |                                 | ministral-3b-latest   | currently points to ministral-3b-2410.<br>World’s best edge model. |  77.4% (instruct) | 63.64% |   |   51.8% |
| Ministral-8B-2410           | 8B           | 8B                  | 24                              | ministral-8b-latest   | currently points to ministral-8b-2410                        |  76.8%  |  72.56%  |    |  56.2%  |
| Codestral-Mamba-7B-v0.1     | 7.3B         | 7.3B                | 16                              | open-codestral-mamba  |       |     75% |  75.61% |     |  39.4%  |
| Mistral-Nemo-Instruct-2407  | 12B          | 12B                 | 28 - bf16,<br>16 - fp8          | open-mistral-nemo     | currently points to open-mistral-nemo-2407  |  67%  | 58.54% |    |  47.4%!  |
| Codestral-22B-v0.1          | 22.2B        | 22.2B               | 60                              |  codestral-latest                     |  currently points to codestral-2405   |  81.1% |  26.83% |    |  37%  |
| Mistral-Small-2409          | 22B          | 22B                 | 60                              | mistral-small-latest                      |  currently points to mistral-small-2409  |  80%  |  70.73%   |    |    60.2%                                                      |
|           |           |                  |                               |                       |                                                              |
| __MODELS I DON'T PLAN TO USE (NOT AN SLM OR IRRELEVANT):__ |
| Mathstral-7B-v0.1           | 7.3B         | 7.3B                | 16                              |                       |                                                              |
| Mixtral-8x22B-v0.3          | 140.6B       | 39.1B               | 300                             |                       |                                                              |
| Pixtral-2409                | 12B          | 12B                 | 28 - bf16,<br>16 - fp8          |                       |                                                              |
| Mistral-Large-Instruct-2407 | 123B         | 123B                | 250                             |                       |                                                              |
| Mistral-Large-Instruct-2411 | 123B         | 123B                | 250                             |                       |                                                              |
| Pixtral-Large-Instruct-2411 | 124B         | 124B                | 250                             |                       |                                                              |

__Notes__:  
* Use the API endpoint name to make API calls.
* Rate limits: 2M tokens/minute, 10B tokens month

__Sources__:
1) https://docs.mistral.ai/getting-started/models/weights/
2) https://docs.mistral.ai/getting-started/models/models_overview/
3) For official HumanEval and MBPP scores, click on each model's blog post link (in link 2)

In [1]:
from human_eval.data import write_jsonl, read_problems, MBPP_TEST
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import time, datetime
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.
    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    text = text.strip()
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [3]:
def get_response(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model = model,
                messages = [{
                        "role": "user",
                        "content": prompt, 
                    }, ]
                )
    return chat_response.choices[0].message.content

In [4]:
def read_jsonl(file_path):
    '''
    Read a JSONL file.
    Parameters:
        file_path (str): The path to the JSONL file.
    Returns:
        List[dict]: A list of dictionaries representing the JSON objects.
    '''
    data = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data

In [5]:
tasks = read_problems(MBPP_TEST)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: 11 <class 'str'>

task_id:
11

test:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

prompt:
Write a python function to remove first and last occurrence of a given character from the string.

canonical_solution:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

entry_point:
None


task_id: 12 <class 'str'>

task_id:
12

test:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

prompt:
Write a function to sort a given matrix in ascending order accordi

In [32]:
task = tasks['15']
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

print(prompt_prefix.format(task['prompt']) + task['test'])

You are an expert Python programmer, and here is your task: Write a function to split a string at lowercase letters. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert split_lowerstring("AbCd")==['bC','d']
assert split_lowerstring("Python")==['y', 't', 'h', 'o', 'n']
assert split_lowerstring("Programming")==['r', 'o', 'g', 'r', 'a', 'm', 'm', 'i', 'n', 'g']


In [7]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [59]:
mistral_7b = 'open-mistral-7b'
mistral_3b = 'ministral-3b-latest'
mistral_8B = 'ministral-8b-latest'
codestral_mamba = 'open-codestral-mamba'
codestral = 'codestral-latest'
nemo = 'open-mistral-nemo'
mistral_small = 'mistral-small-latest'
mixtral = 'open-mixtral-8x7b'

model = mixtral
print(get_response(model, prompt_prefix.format(prompt_prefix)))

Sure, I understand. Here is the Python code that satisfies the given task (assuming the task is to define a function that takes a list of integers and returns the largest product that can be made by multiplying any three integers together):

```python
def largest_product_of_three(numbers):
    numbers.sort()
    n = len(numbers)
    return max(numbers[n-3] * numbers[n-2] * numbers[n-1], numbers[0] * numbers[1] * numbers[n-1])
```

This function first sorts the input list of integers in ascending order. Then, it computes the largest product that can be made by multiplying any three integers together in two different ways: by multiplying the three largest integers, or by multiplying the two smallest integers and the largest integer. The function returns the maximum of these two products.


In [60]:
model

'open-mixtral-8x7b'

In [61]:
model_nickname = 'open-mixtral-8x7b'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'logs/open-mixtral-8x7b_samples_20250104_155006_4734.jsonl'

In [62]:
num_samples_per_task = 1
completions = []
print(f'Model: {model}\n')

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == '15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body['prompt']) + task_body['test']
        try:
            completion  = get_response(model, full_prompt)
        except Exception as e:
            completion = f"Error generating completion:\n{e}"
        
        completion = remove_triple_backticks_at_edges(completion.strip())
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*100, '\n', sep='')

Model: open-mixtral-8x7b

11
Prompt:
You are an expert Python programmer, and here is your task: Write a python function to remove first and last occurrence of a given character from the string. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"
---------------------------------------------------------------------------

Completion:
Here is one way to solve the problem:

```
def remove_occ(s, char):
    # Find the index of the first and last occurrence of the character
    first_index = s.find(char)
    last_index = s.rfind(char)

    # Check if the character appears in the string
    if first_index != -1 and last_index != -1:
        # Remove the first and last occurrence of the character
        s = s[:first_index] + s[first_index+1:last_index] + s[last_index+1:]

    return s
```

Here is how the 

Prompt:
You are an expert Python programmer, and here is your task: Write a function to find sequences of lowercase letters joined with an underscore. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert text_lowercase_underscore("aab_cbbbc")==('Found a match!')
assert text_lowercase_underscore("aab_Abbbc")==('Not matched!')
assert text_lowercase_underscore("Aaab_abbbc")==('Not matched!')
---------------------------------------------------------------------------

Completion:
Here is the Python function that meets the given requirements:

```python
import re

def text_lowercase_underscore(s):
 pattern = r'[a-z]+_[a-z]+'
 if re.search(pattern, s):
 return 'Found a match!'
 else:
 return 'Not matched!'
```

Explanation:

* The function `text_lowercase_underscore` takes a string `s` as input.
* It defines a regular expression pattern using the `re` module to match lowercase letters (`[a-z]`) joined by an undersc

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the maximum sum of elements of list in a list of lists. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert maximum_Sum([[1,2,3],[4,5,6],[10,11,12],[7,8,9]]) == 33
assert maximum_Sum([[0,1,1],[1,1,2],[3,2,1]]) == 6
assert maximum_Sum([[0,1,3],[1,2,1],[9,8,2],[0,1,0],[6,4,8]]) == 19
---------------------------------------------------------------------------

Completion:
Here is one way to write the `maximum_Sum` function in Python:

```
def maximum_Sum(listOfLists):
    return max(sum(sublist) for sublist in listOfLists)
```

This function uses a generator expression to create a list of the sums of each sublist in `listOfLists`, and then takes the maximum value from that list using the `max()` function.

You can use the following tests to verify that the function works correctly:

```
assert maximum_Sum([[1,2,3

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find binomial co-efficient. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert binomial_Coeff(5,2) == 10
assert binomial_Coeff(4,3) == 4
assert binomial_Coeff(3,2) == 3
---------------------------------------------------------------------------

Completion:
Here is the Python function to find binomial coefficient:

```python
def binomial_Coeff(n, k):
    if k > n - k:
        k = n - k
    num = 1
    for i in range(k + 1, n + 1):
        num *= i
    den = 1
    for i in range(1, n - k + 1):
        den *= i
    return num // den
```

This function calculates the binomial coefficient using the formula: n! / [k!(n-k)!], where n! is the factorial of n. The function first checks if k is greater than n-k, and if so, it sets k to n-k. This is because the binomial coefficient is symmetric, i.e. C(n, k) = C(n, n-k). The

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the missing number in a sorted array. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert find_missing([1,2,3,5],4) == 4
assert find_missing([1,3,4,5],4) == 2
assert find_missing([1,2,3,5,6,7],5) == 4
---------------------------------------------------------------------------

Completion:
Here is one way to solve the problem using Python:

```
def find_missing(arr, expected):
  # The expected difference between elements in the sorted array
  diff = next(iter(arr)) - expected
  for i in arr:
    # If the difference between the current element and the expected value is not equal to the expected difference, then the current element is the missing one
    if i - expected != diff:
      return expected + diff
    expected += 1
  # If the loop completes without returning, then the missing number is the last number i

...

Prompt:
You are an expert Python programmer, and here is your task: Write a function to compute binomial probability for the given number. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert binomial_probability(10, 5, 1.0/3) == 0.13656454808718185
assert binomial_probability(11, 6, 2.0/4) == 0.2255859375
assert binomial_probability(12, 7, 3.0/5) == 0.227030335488
---------------------------------------------------------------------------

Completion:
Here is one way to implement the `binomial_probability` function in Python:

```
from math import factorial

def binomial_probability(n, k, p):
    return (factorial(n) / (factorial(k) * factorial(n-k))) * (p**k) * ((1-p)**(n-k))
```

This function uses the formula for the binomial probability mass function, which is:

`P(X=k) = (n! / (k!(n-k)!)) * (p^k) * ((1-p)^(n-k))`

where `n` is the number of trials, `k` is the number of successes, and `p` is the probabil

Prompt:
You are an expert Python programmer, and here is your task: Write a function to calculate a grid of hexagon coordinates where function returns a list of lists containing 6 tuples of x, y point coordinates. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert calculate_polygons(1,1, 4, 4, 3)==[[(-5.0, -4.196152422706632), (-5.0, -0.7320508075688767), (-2.0, 1.0), (1.0, -0.7320508075688767), (1.0, -4.196152422706632), (-2.0, -5.928203230275509), (-5.0, -4.196152422706632)], [(1.0, -4.196152422706632), (1.0, -0.7320508075688767), (4.0, 1.0), (7.0, -0.7320508075688767), (7.0, -4.196152422706632), (4.0, -5.928203230275509), (1.0, -4.196152422706632)], [(7.0, -4.196152422706632), (7.0, -0.7320508075688767), (10.0, 1.0), (13.0, -0.7320508075688767), (13.0, -4.196152422706632), (10.0, -5.928203230275509), (7.0, -4.196152422706632)], [(-2.0, 1.0000000000000004), (-2.0, 4.464101615137755), (1.0, 6.1961524227066

Prompt:
You are an expert Python programmer, and here is your task: Write a function to convert the given binary tuple to integer. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert binary_to_integer((1, 1, 0, 1, 0, 0, 1)) == '105'
assert binary_to_integer((0, 1, 1, 0, 0, 1, 0, 1)) == '101'
assert binary_to_integer((1, 1, 0, 1, 0, 1)) == '53'
---------------------------------------------------------------------------

Completion:
Here is the Python function to convert the given binary tuple to integer:

```python
def binary_to_integer(binary_tuple):
    return int(''.join(map(str, binary_tuple)), 2)
```

This function uses the `join()` method to convert the binary tuple into a string, and then uses the `int()` function to convert the binary string to an integer. The second argument of the `int()` function is set to `2` to specify that the string is a binary number.

Time elapsed: 1.2596 seconds


495
Prompt

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find remainder of two numbers. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert find(3,3) == 0
assert find(10,3) == 1
assert find(16,5) == 1
---------------------------------------------------------------------------

Completion:
Here is the Python function to find the remainder of two numbers:

```python
def find(num1, num2):
    return num1 % num2
```

This function takes two numbers as input arguments and returns the remainder of those two numbers using the modulus operator `%`. The above tests provided in the prompt will pass for this function.

Time elapsed: 1.0721 seconds


503
Prompt:
You are an expert Python programmer, and here is your task: Write a function to add consecutive numbers of a given list. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should s

Prompt:
You are an expert Python programmer, and here is your task: Write a function to check if the common elements between two given lists are in the same order or not. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
assert same_order(["red","pink","green","white","black"],["white","orange","pink","black"])==False
assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
---------------------------------------------------------------------------

Completion:
def same_order(list1, list2):
    common_elements = [element for element in list1 if element in list2]
    return all(common_elements[i] == list2[i] for i in range(len(common_elements)))
```
This function first finds the common elements between the two lists using a list comprehension. Then, it checks if those common elem

In [12]:
# if the above cell has not been run
#file_name = 'logs2/solar_10B_samples_20241127_232205_5243.jsonl'

In [65]:
evaluate_functional_correctness(file_name, k=[1], mode='mbpp')

Reading samples...


500it [00:00, 7644.19it/s]


Running test suites...


100%|██████████████████████████████████████████████████████| 500/500 [00:32<00:00, 15.58it/s]


Writing results to logs/open-mixtral-8x7b_samples_20250104_155006_4734.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████| 500/500 [00:00<00:00, 65411.31it/s]


{'pass@1': 0.0}

## Getting Model Temperature and Top_p
According to [Mistral docs](https://docs.mistral.ai/api/#tag/chat/operation/chat_completion_v1_chat_completions_post):
* Default top_p is 1
* To find out about the default temperature, one neexs to call the `/models` endpoint to retrieve the appropriate value.

In [70]:
# calling the /models endpoint
import requests

api_key = os.environ["MISTRAL_API_KEY"]

# Correct API endpoint
url = "https://api.mistral.ai/v1/models"

# Headers with the correct authorization format
headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for HTTP 4xx/5xx codes
    print("Response:")
    print(response.json())
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


Response:
{'object': 'list', 'data': [{'id': 'ministral-3b-2410', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-latest'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-3b-latest', 'object': 'model', 'created': 1736121321, 'owned_by': 'mistralai', 'capabilities': {'completion_chat': True, 'completion_fim': False, 'function_calling': True, 'fine_tuning': True, 'vision': False}, 'name': 'ministral-3b-2410', 'description': 'Official ministral-3b-2410 Mistral AI model', 'max_context_length': 131072, 'aliases': ['ministral-3b-2410'], 'deprecation': None, 'default_model_temperature': 0.3, 'type': 'base'}, {'id': 'ministral-8b-2410', 'object': 'model', 'cr

In [72]:
response.json()

{'object': 'list',
 'data': [{'id': 'ministral-3b-2410',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-latest'],
   'deprecation': None,
   'default_model_temperature': 0.3,
   'type': 'base'},
  {'id': 'ministral-3b-latest',
   'object': 'model',
   'created': 1736121321,
   'owned_by': 'mistralai',
   'capabilities': {'completion_chat': True,
    'completion_fim': False,
    'function_calling': True,
    'fine_tuning': True,
    'vision': False},
   'name': 'ministral-3b-2410',
   'description': 'Official ministral-3b-2410 Mistral AI model',
   'max_context_length': 131072,
   'aliases': ['ministral-3b-2410'],
   'deprecation': None,
   'default_mode

In [79]:
for i in response.json()['data']:
    print(f"Model: {i['id']}. Temperature: {i['default_model_temperature']}")

Model: ministral-3b-2410. Temperature: 0.3
Model: ministral-3b-latest. Temperature: 0.3
Model: ministral-8b-2410. Temperature: 0.3
Model: ministral-8b-latest. Temperature: 0.3
Model: open-mistral-7b. Temperature: 0.7
Model: mistral-tiny. Temperature: 0.7
Model: mistral-tiny-2312. Temperature: 0.7
Model: open-mistral-nemo. Temperature: 0.3
Model: open-mistral-nemo-2407. Temperature: 0.3
Model: mistral-tiny-2407. Temperature: 0.3
Model: mistral-tiny-latest. Temperature: 0.3
Model: open-mixtral-8x7b. Temperature: 0.7
Model: mistral-small. Temperature: 0.7
Model: mistral-small-2312. Temperature: 0.7
Model: open-mixtral-8x22b. Temperature: 0.7
Model: open-mixtral-8x22b-2404. Temperature: 0.7
Model: mistral-small-2402. Temperature: 0.7
Model: mistral-small-2409. Temperature: 0.7
Model: mistral-small-latest. Temperature: 0.7
Model: mistral-medium-2312. Temperature: 0.7
Model: mistral-medium. Temperature: 0.7
Model: mistral-medium-latest. Temperature: 0.7
Model: mistral-large-2402. Temperature